In [ ]:
!pip install -r ../requirements.txt

In [109]:
import itertools
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import r2_score, mean_squared_error
import joblib
import pandas as pd
import os
from tqdm import tqdm

# Overall Settings

In [119]:
DATA_PATH = "../data/data.csv"
MODEL_DIR = "../saved_models_filtered_rf"
NUMBER_OF_ROWS = 100000
N_ESTIMATORS = 100
SEED = 42
NUMBER_OF_THREADS = os.cpu_count()
os.makedirs(MODEL_DIR, exist_ok=True)

# Define Inputs
We decided to have one mandatory Input with the "CCSR Procedure Code" and many other optional inputs.
For each combination of the optional inputs and the one mandatory input we have to train a random forrest models.

In [120]:
optional_features = ['Age Group', 'Gender', 'Race', 'Ethnicity']
base_feature = ['CCSR Procedure Code', 'Type of Admission']
all_combinations = []

for r in range(len(optional_features) + 1):
    for combo in itertools.combinations(optional_features, r):
        all_combinations.append(base_feature + list(combo))

# Define Outputs

In [121]:
targets = ['Total Costs', 'Length of Stay', 'APR Risk of Mortality']

# Loading the Data
Now we load our preprocessed data and clean some parts up.
We also encode the "APR Risk of Mortality"

In [122]:
df = pd.read_csv(DATA_PATH, dtype=str, low_memory=False, nrows=NUMBER_OF_ROWS)

# make numbers correct
df['Total Costs'] = df['Total Costs'].astype(float)
#df['Total Charges'] = df['Total Charges'].astype(float)
# Replace "120 +" with 140 and convert to float
df['Length of Stay'] = df['Length of Stay'].replace("120 +", "140").astype(float)

# Encode the risk of mortality
mortality_encoder = LabelEncoder()
df['APR Risk of Mortality'] = mortality_encoder.fit_transform(df['APR Risk of Mortality'])

# Print the number of loaded rows
print(f"Number of loaded rows: {len(df)}")

Number of loaded rows: 100000


In [123]:
# Python
def filter_outliers(df, feature):
    median = df[feature].median()
    lower_bound = median * 0.5
    upper_bound = median * 1.5
    return df[(df[feature] >= lower_bound) & (df[feature] <= upper_bound)]

filtered_dfs = []

# Iterate over unique combinations of procedure code and type of admission
for procedure_code, admission_type in df[['CCSR Procedure Code', 'Type of Admission']].drop_duplicates().itertuples(index=False):
    subset = df[(df['CCSR Procedure Code'] == procedure_code) & (df['Type of Admission'] == admission_type)]
    #for feature in targets:
    #    subset = filter_outliers(subset, feature)
    subset = filter_outliers(subset, 'Total Costs')
    filtered_dfs.append(subset)

# Combine all filtered subsets
filtered_df = pd.concat(filtered_dfs, ignore_index=True)
df = filtered_df

# Print the number of rows after filtering
print(f"Number of rows after filtering: {len(filtered_df)}")

Number of rows after filtering: 60059


# Define the model training function

In [115]:
def train_model(features):
    try:
        # OneHot-Encoding der Features
        encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
        X_encoded = encoder.fit_transform(df[features])
        X_df = pd.DataFrame(X_encoded, columns=encoder.get_feature_names_out(features))

        y_df = df[targets].reset_index(drop=True)

        # Move data to GPU
        # dtrain = xgb.DMatrix(X_df, label=y_df, device='cuda')

        # XGBoost-Konfiguration für maximale Performance
        base_model = xgb.XGBRegressor(
            n_estimators=N_ESTIMATORS,             # z. B. 100 oder 500
            tree_method='hist',                # GPU!
            booster='gbtree',                      # Tree Booster
            device = 'cpu',                        # GPU!
            max_depth=5,                          # Tiefer = komplexer
            subsample=0.8,                         # Bagging
            colsample_bytree=0.8,                  # Feature Sampling
            learning_rate=0.1,                     # kleiner bei mehr Estimators
            n_jobs = -1  # nutzt alle CPU-Kerne
        )

        # MultiOutputRegressor für 4 Zielspalten
        model = MultiOutputRegressor(base_model)
        model.fit(X_df.to_numpy(), y_df.to_numpy())

        # Vorhersage und Bewertung
        y_pred = model.predict(X_df)

        scores = {}
        for i, target in enumerate(targets):
            scores[f"{target}_r2"] = r2_score(y_df.iloc[:, i], y_pred[:, i])
            scores[f"{target}_mse"] = mean_squared_error(y_df.iloc[:, i], y_pred[:, i])

        # Modell speichern
        model_name = f"{'__'.join(f.replace(' ', '_') for f in features)}.pkl"
        model_path = os.path.join(MODEL_DIR, model_name)

        joblib.dump({
            "model": model,
            "features": features,
            "encoder": encoder,
            "target_columns": targets,
            "mortality_encoder": mortality_encoder
        }, model_path)

        return {
            "features": features,
            "model_path": model_path,
            **scores
        }

    except Exception as e:
        print(f"Error training model for features {features}: {e}", flush=True)
        return None

# Real Random Forest Model Training

In [124]:
def train_model(features):
    try:
        # OneHot-Encoding der Features
        encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
        X_encoded = encoder.fit_transform(df[features])
        X_df = pd.DataFrame(X_encoded, columns=encoder.get_feature_names_out(features))

        y_df = df[targets].reset_index(drop=True)

        # Random Forest-Konfiguration für maximale Performance
        base_model = RandomForestRegressor(
            n_estimators=N_ESTIMATORS,  # Number of trees
            max_depth=5,               # Maximum depth of the trees
            n_jobs=-1,                 # Use all CPU cores
            random_state=SEED          # Ensure reproducibility
        )

        # MultiOutputRegressor für 4 Zielspalten
        model = MultiOutputRegressor(base_model)
        model.fit(X_df.to_numpy(), y_df.to_numpy())

        # Vorhersage und Bewertung
        y_pred = model.predict(X_df.to_numpy())

        scores = {}
        for i, target in enumerate(targets):
            scores[f"{target}_r2"] = r2_score(y_df.iloc[:, i], y_pred[:, i])
            scores[f"{target}_mse"] = mean_squared_error(y_df.iloc[:, i], y_pred[:, i])

        # Modell speichern
        model_name = f"{'__'.join(f.replace(' ', '_') for f in features)}.pkl"
        model_path = os.path.join(MODEL_DIR, model_name)

        joblib.dump({
            "model": model,
            "features": features,
            "encoder": encoder,
            "target_columns": targets,
            "mortality_encoder": mortality_encoder
        }, model_path)

        return {
            "features": features,
            "model_path": model_path,
            **scores
        }

    except Exception as e:
        print(f"Error training model for features {features}: {e}", flush=True)
        return None

In [117]:
from sklearn.model_selection import train_test_split
from keras.src.layers import Dense
from keras.src.optimizers import Adam
from keras import Sequential, Input


def train_model(features):
    try:
        # OneHot-Encoding of features
        encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
        X_encoded = encoder.fit_transform(df[features])
        X_df = pd.DataFrame(X_encoded, columns=encoder.get_feature_names_out(features))
        
        print(X_df.shape)

        y_df = df[targets].reset_index(drop=True)

        # Ensure consistent sample sizes
        if len(X_df) != len(y_df):
            raise ValueError(f"Inconsistent sample sizes: {len(X_df)} features, {len(y_df)} targets")

        # Normalize targets
        scaler = StandardScaler()
        y_scaled = scaler.fit_transform(y_df)

        # Split data into training and validation sets
        X_train, X_val, y_train, y_val = train_test_split(X_df, y_scaled, test_size=0.2, random_state=SEED)

        # Build neural network
        model = Sequential([
            Input(shape=(X_train.shape[1],)),  # Define input shape
            Dense(128, activation='relu'),
            Dense(64, activation='relu'),
            Dense(len(targets), activation='linear')  # Output layer for regression
        ])

        # Compile model
        model.compile(optimizer=Adam(learning_rate=0.01), loss='mse', metrics=['mae'])

        # Train model
        model.fit(X_train.to_numpy(), y_train, epochs=150, batch_size=32, validation_data=(X_val.to_numpy(), y_val), verbose=1)

        # Predictions and evaluation
        y_pred_scaled = model.predict(X_val.to_numpy())
        y_pred = scaler.inverse_transform(y_pred_scaled)

        # Save model
        model_name = f"{'__'.join(f.replace(' ', '_') for f in features)}.keras"
        model_path = os.path.join(MODEL_DIR, model_name)
        model.save(model_path)

        joblib.dump({
            "features": features,
            "encoder": encoder,
            "target_columns": targets,
            "scaler": scaler
        }, model_path.replace('.keras', '_metadata.pkl'))

        return {
            "features": features,
            "model_path": model_path,
        }

    except Exception as e:
        print(f"Error training model for features {features}: {e}", flush=True)
        return None

# Use Multithreading for the model training

In [118]:
#results = Parallel(n_jobs=NUMBER_OF_THREADS)(
#    delayed(train_model)(feature_comb) for feature_comb in tqdm(all_combinations, desc="Training Models")
#)


results = []
for feature_comb in tqdm(all_combinations, desc="Training Models"):
    result = train_model(feature_comb)
    if result is not None:
        results.append(result)

# Save summary
results_df = pd.DataFrame(results)
print(results_df)
results_df.to_csv(os.path.join(MODEL_DIR, "model_overview.csv"), index=False)
print("\n📦 Models saved in:", MODEL_DIR)
print("📄 Summary saved as: model_overview.csv")

Training Models:   0%|          | 0/16 [00:00<?, ?it/s]

(60059, 231)
Epoch 1/150
1502/1502 ━━━━━━━━━━━━━━━━━━━━ 1s 746us/step - loss: 0.7156 - mae: 0.4935 - val_loss: 0.4969 - val_mae: 0.4619
Epoch 2/150
1502/1502 ━━━━━━━━━━━━━━━━━━━━ 1s 696us/step - loss: 0.7958 - mae: 0.4718 - val_loss: 0.5024 - val_mae: 0.4653
Epoch 3/150
1502/1502 ━━━━━━━━━━━━━━━━━━━━ 1s 700us/step - loss: 0.7653 - mae: 0.4672 - val_loss: 0.4933 - val_mae: 0.4672
Epoch 4/150
1502/1502 ━━━━━━━━━━━━━━━━━━━━ 1s 700us/step - loss: 0.6362 - mae: 0.4658 - val_loss: 0.5244 - val_mae: 0.4536
Epoch 5/150
1502/1502 ━━━━━━━━━━━━━━━━━━━━ 1s 834us/step - loss: 0.5541 - mae: 0.4576 - val_loss: 0.5627 - val_mae: 0.4609
Epoch 6/150
1502/1502 ━━━━━━━━━━━━━━━━━━━━ 1s 694us/step - loss: 0.5910 - mae: 0.4616 - val_loss: 0.4986 - val_mae: 0.4598
Epoch 7/150
1502/1502 ━━━━━━━━━━━━━━━━━━━━ 1s 701us/step - loss: 0.5028 - mae: 0.4525 - val_loss: 0.4893 - val_mae: 0.4533
Epoch 8/150
1502/1502 ━━━━━━━━━━━━━━━━━━━━ 1s 693us/step - loss: 0.5083 - mae: 0.4573 - val_loss: 0.4948 - val_mae: 0.4578
Epo

Training Models:   0%|          | 0/16 [00:12<?, ?it/s]


KeyboardInterrupt: 